In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import keras.utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.regularizers as kr
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.losses as ls
import matplotlib.pyplot as plt
import matplotlib
import sys
import math as m
np.set_printoptions(threshold=sys.maxsize)
%matplotlib qt

Using TensorFlow backend.


In [2]:
###Vengono caricati i dati dal file, questa volta non viene fatta nessuna selezione a priori
###a parte quella per numero di elementi non nulli.

d=pd.read_csv('./Settings/Data/Default/DEFGBData.csv')
index=np.load('./Settings/valindgb.npy')
valin=np.load('./Settings/valingb.npy')
valout=np.load('./Settings/valoutgb.npy')
index=index.tolist()
nancount=[]
ind=[]
for i in range(0,d.shape[0]):
    nancount.append(d.iloc[i,12:].isnull().sum())
    if nancount[i]==0:
        ind.append(i)
    else:
        pass
data=d.iloc[ind,:]
data=data.set_index(['GES_FLD'])

data=data.reset_index()   
data1=data.values

dataset=data.values[:,12:].astype(float)
datafei=data.values[:,9].astype(float)
dataset=np.nan_to_num(dataset)
datafei=np.nan_to_num(datafei)
dataset=np.c_[dataset,datafei]

asun=[6.24,7.60,6.47,6.34,3.15,3.15,4.95,3.93,5.64,5.64,7.50,7.50,4.99,6.22,2.21]
for i in range(0,dataset.shape[0]):
    for j in range(0,dataset.shape[1]-1):
        if j==10:
            pass
        else:
            dataset[i,j]=(dataset[i,j]-asun[j])-(dataset[i,10]-asun[10])

dataset=np.delete(dataset,10,1)
invar=dataset

In [24]:
###Plot combinato di loss e accuracy
valacc=np.load('./Settings/valaccuracygb.npy')
loss=np.load('./Settings/trainlossgb.npy')
valloss=np.load('./Settings/vallossgb.npy')

y1 = np.random.random_sample(len(valacc))
y2 = np.random.normal(0.0,0.2,len(valacc))
jittery=valacc+0.1*y2
title='Learning Curves, 5 OCs'
plt.close()
fig, (ax1, ax2) = plt.subplots(figsize=(11.5, 12),nrows=2, sharex=True)
plt.rcParams['xtick.labelsize']=22
plt.rcParams['ytick.labelsize']=22
plt.rcParams['axes.labelsize']=25
plt.rc('legend',fontsize=22)

plt.subplots_adjust(wspace=0, hspace=0)
ax1.plot(loss, label='Training set')
ax1.plot(valloss, label='Validation set')
ax2.scatter(list(range(1,jittery.shape[0]+1)),jittery,marker='.',color='blue',facecolor='None',label='Validation accuracy')

plt.suptitle(title,fontsize=50)
plt.xlabel('Epochs')
ax1.set_ylabel('Loss')
ax2.set_ylabel('Accuracy')
ax1.legend(loc='upper right')
ax2.legend(loc='lower right')
plt.show()
plt.tight_layout()
plt.savefig('./Results/learningcurves5oc.png')

In [3]:
###Nuova NN. Vengono caricati i files preparati con il training e associati ai rispettivi layers.

model=Sequential()
model.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.04)))
model.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.03)))
model.add(Dense(3,activation='softsign'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
w0=np.load("./Settings/Weights/gbl0.npy")
b0=np.load("./Settings/Weights/gbl0bias.npy")
w1=np.load("./Settings/Weights/gbl1.npy")
b1=np.load("./Settings/Weights/gbl1bias.npy")
w2=np.load("./Settings/Weights/gbl2.npy")
b2=np.load("./Settings/Weights/gbl2bias.npy")
w3=np.load("./Settings/Weights/gbl3.npy")
b3=np.load("./Settings/Weights/gbl3bias.npy")
model.layers[0].set_weights([w0,b0])
model.layers[1].set_weights([w1,b1])
model.layers[2].set_weights([w2,b2])

In [4]:
###Viene fatta una run con tutte le stelle, si selezionano quali visualizzare direttamente nei grafici.

out=model.predict(invar)

In [5]:
dummyout=np_utils.to_categorical(valout)
dummyout=dummyout[0]
model2=Sequential()
model2.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model2.add(Dense(3,activation='softsign'))
model2.add(Dense(dummyout.shape[1],activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model2.layers[0].set_weights([w0,b0])
model2.layers[1].set_weights([w1,b1])
model2.layers[2].set_weights([w2,b2])
model2.layers[3].set_weights([w3,b3])

out2=model2.predict(valin)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dummyout.argmax(axis=1),out2.argmax(axis=1))

#format out: [precision, recall, accuracy, f-score]
outm=np.zeros((5,4))
for i in range(5):
    tp=cm[i,i]
    fp=sum(cm[i,:])-cm[i,i]
    fn=sum(cm[:,i])-cm[i,i]
    tn=sum(sum(cm))-tp-fp-fn
    outm[i,0]=tp/(tp+fp)
    outm[i,1]=tp/(tp+fn)
    outm[i,2]=(tp+tn)/(tp+tn+fp+fn)
    outm[i,3]=2*((outm[i,0]*outm[i,1])/(outm[i,0]+outm[i,1]))

values=np.mean(outm,axis=0)
prec=values[0]
rec=values[1]
acc=values[2]
fsc=values[3]
print('Precision = ',prec,'; Recall = ',rec,'; Accuracy = ',acc,'; F-score = ',fsc)
#acc=(cm[0,0]+cm[1,1]+cm[2,2]+cm[3,3]+cm[4,4])/sum(sum(cm))
#print ('Accuracy = ',round(acc,3))

Precision =  0.7992546583850931 ; Recall =  0.806083916083916 ; Accuracy =  0.9370786516853933 ; F-score =  0.8004046746903889


In [9]:
###Primo grafico con OC nuovi (NON QUELLI DEL TRAINING)
ob1='NGC6802'
ob2='NGC6005'
ob3='NGC6253'
ob4='Br31'
ob5=''

import matplotlib.gridspec as gridspec
c1=0
c2=0
c3=0
c4=0
c5=0
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)

fig=plt.figure(figsize=(12,10))
fig.suptitle('Abundances Space, Unknown OCs', fontsize=20)
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron 1',fontsize=15)
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron 3',fontsize=15)
ax2.set_xlabel('Output Neuron 2',fontsize=15)
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron 3',fontsize=15)
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron 2',fontsize=15)
ax3.set_ylabel('Output Neuron 1',fontsize=15)
ax3.yaxis.set_label_position('right')
for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob1:
        if c1==0:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+',label=ob1)
            c1=1
        elif c1==1:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif data1[i,0]==ob2:
        if c2==0:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1',label=ob2)
            c2=1
        elif c2==1:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='blue',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='blue',marker='1')
    elif data1[i,0]==ob3:
        if c3==0:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4',label=ob3)
            c3=1
        elif c3==1:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='green',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='green',marker='4')
    elif data1[i,0]==ob4:
        if c4==0:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.',label=ob4)
            c4=1
        elif c4==1:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='black',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='black',marker='.')
    elif data1[i,0]==ob5:
        if c5==0:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x',label=ob5)
            c5=1
        elif c5==1:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='cyan',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='cyan',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0,fontsize=15)
fig.text(0.36,0.25,'Precision = ',fontsize=15,ha='right')
fig.text(0.36,0.25,round(prec,2),fontsize=15)
fig.text(0.36,0.22,'Recall = ',fontsize=15,ha='right')
fig.text(0.36,0.22,round(rec,2),fontsize=15)
fig.text(0.36,0.19,'Accuracy = ',fontsize=15,ha='right')
fig.text(0.36,0.19,round(acc,2),fontsize=15)
fig.text(0.36,0.16,'F-score = ',fontsize=15,ha='right')
fig.text(0.36,0.16,round(fsc,2),fontsize=15)
plt.tight_layout()
plt.savefig('./Results/CT5OC(unknown).png')

C:\Users\giang\anaconda3\lib\site-packages\ipykernel_launcher.py:90: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [8]:
###Secondo grafico, con OC del training
ob01='NGC2420'
ob02='Rup134'
ob03='Trumpler20'
ob04='NGC2243'
ob05='Trumpler23'

import matplotlib.gridspec as gridspec
c01=0
c02=0
c03=0
c04=0
c05=0
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure(figsize=(12,10))
fig.suptitle('Abundances Space, Validation Set OCs', fontsize=20)
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('Output Neuron 1',fontsize=15)
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('Output Neuron 3',fontsize=15)
ax2.set_xlabel('Output Neuron 2',fontsize=15)
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('Output Neuron 3',fontsize=15)
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('Output Neuron 2',fontsize=15)
ax3.set_ylabel('Output Neuron 1',fontsize=15)
ax3.yaxis.set_label_position('right')

for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob01 and i in index:
        if c01==0:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+',label=ob01)
            c01=1
        elif c01==1:
            ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif data1[i,0]==ob02 and i in index:
        if c02==0:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1',label=ob02)
            c02=1
        elif c02==1:
            ax1.scatter(out[i,0],out[i,2],color='blue',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='blue',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='blue',marker='1')
    elif data1[i,0]==ob03 and i in index:
        if c03==0:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4',label=ob03)
            c03=1
        elif c03==1:
            ax1.scatter(out[i,0],out[i,2],color='green',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='green',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='green',marker='4')
    elif data1[i,0]==ob04 and i in index:
        if c04==0:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.',label=ob04)
            c04=1
        elif c04==1:
            ax1.scatter(out[i,0],out[i,2],color='black',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='black',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='black',marker='.')
    elif data1[i,0]==ob05 and i in index:
        if c05==0:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x',label=ob05)
            c05=1
        elif c05==1:
            ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='cyan',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='cyan',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0,fontsize=15)
fig.text(0.36,0.22,'Precision = ',fontsize=15,ha='right')
fig.text(0.36,0.22,round(prec,2),fontsize=15)
fig.text(0.36,0.19,'Recall = ',fontsize=15,ha='right')
fig.text(0.36,0.19,round(rec,2),fontsize=15)
fig.text(0.36,0.16,'Accuracy = ',fontsize=15,ha='right')
fig.text(0.36,0.16,round(acc,2),fontsize=15)
fig.text(0.36,0.13,'F-score = ',fontsize=15,ha='right')
fig.text(0.36,0.13,round(fsc,2),fontsize=15)
plt.tight_layout()
plt.savefig('./Results/CT5OC(validation).png')

C:\Users\giang\anaconda3\lib\site-packages\ipykernel_launcher.py:87: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [ ]:
###Prova plot 3d fatta per i nuovi OC

plt.close()
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
for i in range(0,dataset.shape[0]):
    if data1[i,0]==ob01 and i in index:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='red',marker='+')
    elif data1[i,0]==ob02 and i in index:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='blue',marker='1')
    elif data1[i,0]==ob03 and i in index:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='green',marker='4')
    elif data1[i,0]==ob04 and i in index:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='black',marker='.')
    elif data1[i,0]==ob05 and i in index:
        ax.scatter(out[i,0],out[i,1],out[i,2],color='cyan',marker='x')